## Circuit cutting with automatic cut finding using the Circuit Knitting Toolbox

### Import relevant modules

In [1]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, Options, Session, Sampler

from circuit_knitting_toolbox.circuit_cutting import WireCutter

### Create a circuit to cut

In [2]:
qc = QuantumCircuit(5)
for i in range(5):
    qc.h(i)
qc.cx(0, 1)
for i in range(2, 5):
    qc.t(i)
qc.cx(0, 2)
qc.rx(np.pi / 2, 4)
qc.rx(np.pi / 2, 0)
qc.rx(np.pi / 2, 1)
qc.cx(2, 4)
qc.t(0)
qc.t(1)
qc.cx(2, 3)
qc.ry(np.pi / 2, 4)
for i in range(5):
    qc.h(i)

qc.draw()

┌───┐                ┌─────────┐   ┌───┐   ┌───┐
q_0: ┤ H ├──■───────■─────┤ Rx(π/2) ├───┤ T ├───┤ H ├
     ├───┤┌─┴─┐     │     ├─────────┤   ├───┤   ├───┤
q_1: ┤ H ├┤ X ├─────┼─────┤ Rx(π/2) ├───┤ T ├───┤ H ├
     ├───┤├───┤   ┌─┴─┐   └─────────┘   └───┘   ├───┤
q_2: ┤ H ├┤ T ├───┤ X ├────────■──────────■─────┤ H ├
     ├───┤├───┤   └───┘        │        ┌─┴─┐   ├───┤
q_3: ┤ H ├┤ T ├────────────────┼────────┤ X ├───┤ H ├
     ├───┤├───┤┌─────────┐   ┌─┴─┐   ┌──┴───┴──┐├───┤
q_4: ┤ H ├┤ T ├┤ Rx(π/2) ├───┤ X ├───┤ Ry(π/2) ├┤ H ├
     └───┘└───┘└─────────┘   └───┘   └─────────┘└───┘

## Set up the Qiskit runtime session

In [3]:
service = QiskitRuntimeService(
    channel="ibm_quantum",
    instance="system-request/7-8-22-access/main",
    token="af0c18b11e3ac4f44582dcccc09b543c998e3caacb5bc151f059ef098195be53f1e33c5c71933bb3c1e68fc8173fc67e5433f6a2a0d7457ee875b6712c2cee1a",
)
session = Session(service=service, backend="ibmq_qasm_simulator")

### Find cuts that match our criteria and evaluate the subcircuits within a Qiskit Session context

In [4]:
options = Options(resilience_level=1, optimization_level=3, execution={"shots": 8192})
with Sampler(session=session, options=options) as sampler:
    cutter = WireCutter(qc, sampler)
    cuts = cutter.decompose(
        method="automatic",
        max_subcircuit_width=6,
        max_cuts=2,
        num_subcircuits=[2],
    )
    subcircuit_instance_probabilities = cutter.evaluate(cuts)

/Users/caleb/opt/anaconda3/envs/ckt/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: The BaseSampler.__enter__ method is deprecated as of Qiskit Terra 0.22.0 and will be removed no sooner than 3 months after the releasedate. BaseSampler should be initialized directly.
  


Exporting as a LP file to let you check the model that will be solved :  inf <class 'float'>
Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               300
Tried aggregator 3 times.
MIP Presolve eliminated 20 rows and 5 columns.
MIP Presolve modified 4 coefficients.
Aggregator did 24 substitutions.
Reduced MIP has 44 rows, 19 columns, and 131 nonzeros.
Reduced MIP has 15 binaries, 4 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.31 ticks)
Found incumbent of value 1.000000 after 0.01 sec. (0.44 ticks)
Probing fixed 0 vars, tightened 2 bounds.
Probing time = 0.00 sec. (0.08 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 0 rows and 3 columns.
Reduced MIP has 44 rows, 16 columns, and 131 nonzeros.
Reduced MIP has 12 binaries, 4 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.08 ticks)
Probing time = 0.00 sec. (0.07 ticks)
Tried aggregator 1 time.
Detecti

/Users/caleb/opt/anaconda3/envs/ckt/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: The BaseSampler.__exit__ method is deprecated as of Qiskit Terra 0.22.0 and will be removed no sooner than 3 months after the releasedate. BaseSampler should be initialized directly.
  # Remove the CWD from sys.path while we load stuff.


## Recompose the circuit and verify the error between the full and cut circuit distributions is within tolerance

In [5]:
ordered_probabilities = cutter.recompose(
    subcircuit_instance_probabilities, cuts, num_threads=4
)
metrics = cutter.verify(ordered_probabilities)
print(metrics)

{'nearest': {'chi2': 0.0011238926948706916, 'Mean Squared Error': 2.109926531952161e-06, 'Mean Absolute Percentage Error': 6.22471559313873, 'Cross Entropy': 2.600820024333325, 'HOP': 0.8988178372383118}, 'naive': {'chi2': 0.0011238926948706916, 'Mean Squared Error': 2.109926531952161e-06, 'Mean Absolute Percentage Error': 6.22471559313873, 'Cross Entropy': 2.600820024333325, 'HOP': 0.8988178372383118}}
